Connected to .env (Python 3.10.16)


### 什么是 LlamaIndex？`

`LlamaIndex` 是一个用于 LLM 应用程序的数据框架，用于注入，结构化，并访问私有或特定领域数据。


LlamaIndex 是一个数据框架，用于构建 LLM（大型语言模型）应用程序。

它提供了一套工具，帮助开发者连接自定义数据源与大型语言模型，从而创建能够利用私有或特定领域数据的智能应用


### 5 大核心工具

![llamaindex应运而生](./images/image.png)

`LlamaIndex` 提供了 5 大核心工具：

- Data connectors
- Data indexes
- Engines
- Data agents
- Application integrations


- Data Connectors（数据连接器），数据连接器用于从各种来源加载数据到 LlamaIndex 中。
- Data Indexes（数据索引），索引是对原始数据的结构化表示，使 LLM 能够高效访问和查询。
- Engines（引擎），引擎是与索引交互的高级接口，引擎简化了应用程序与索引的交互。
- Data Agents（数据代理），数据代理是自主的 LLM 助手，代理可以自主决定何时检索信息、使用工具或直接回答。
- Application Integrations（应用集成），与各种应用框架和平台的集成，LangChain、FastAPI、LlamaHub 等。


快速使用

`pip install llama-index`


In [57]:
# 默认llamaindex 使用 openaai的kei，这里我们使用本地部署的ollama
# pip install llama-index-llms-ollama 用来调用本地部署的ollama
# pip install llama-index-embeddings-ollama 用来调用本地部署的ollama的embedding

from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

llm = Ollama(
  model="deepseek-r1:8b", 
  base_url="http://localhost:11434",
  system_prompt="你是一个助手，请直接回答问题，不要输出任何形式的思考过程，不要使用<think>或</think>标签。",
  # 可以添加其他参数来控制输出
  temperature=0.1,  # 降低随机性
)

Settings.llm = llm  # 设置 llama index 使用本地部署的ollama

# 测试是否连接成功
response = Settings.llm.complete("你好，准备好了吗?")
# 手动过滤掉可能的思考过程和多余空格
filtered_response = response.text.replace("<think>", "").replace("</think>", "").strip()
print(filtered_response)

你好！我已经准备好了，随时可以帮助你。请告诉我你需要什么帮助。


```python
我们可以设置 一些参数
'''
- 控制每个文本块包含的文本量
- 影响索引的粒度和检索精度
- 较小的块大小使检索更精确，但可能丢失上下文
- 较大的块大小保留更多上下文，但可能包含过多无关信息
'''
Settings.chunk_size = 512 # 默认 1024  将文档分割成的每个文本块的大小

'''
- 控制文本块之间的重叠度
- 防止句子或段落在分块时被截断
- 帮助保留跨块的上下文信息
- 提高检索时的连贯性
'''
Settings.chunk_overlap = 32 # 默认 200

'''
参数选择的影响
- 较小的 chunk_size + 较小的 overlap ：

  - 优点：更精确的检索，节省存储空间和嵌入成本
  - 缺点：可能丢失上下文，影响理解长句或复杂概念
- 较大的 chunk_size + 较大的 overlap ：

  - 优点：保留更多上下文，更好地理解复杂内容
  - 缺点：增加存储和计算成本，可能检索到更多不相关内容
'''
```


In [48]:
# 设置 embedding 模型
# 这里我们依然使用本地ollama模型
from llama_index.embeddings.ollama import OllamaEmbedding

model_name = "imcurie/bge-large-en-v1.5"
base_url = "http://localhost:11434"
Settings.embed_model = OllamaEmbedding(model_name,base_url,embed_batch_size=100)

Settings.chunk_size = 512
Settings.chunk_overlap = 32

print(Settings.chunk_size, Settings.chunk_overlap)
print(Settings.embed_model.model_name)

512 32
imcurie/bge-large-en-v1.5


In [59]:

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# 加载文档
documents = SimpleDirectoryReader("data").load_data()

# 创建索引
index = VectorStoreIndex.from_documents(documents)

# 创建查询引擎
query_engine = index.as_query_engine()

# 查询数据
response = query_engine.query("首开股份因何被中国证监会处罚？")
print(response)

<think>
首先，我需要理解用户的问题：“首开股份因何被中国证监会处罚？”根据提供的上下文信息，首开股份在2021年存货减值测试中高估了子公司存货可变现净值，导致少计提存货跌价准备和资产减值损失4.05亿元。这一错误使得公司在2022年5月至2023年5月期间的发行文件和银行间债券市场披露的年度报告中存在错报。因此，中国证监会对首开股份进行了行政处罚，包括警告并处以150万元的罚款。
</think>

首开股份因在2021年存货减值测试中高估下属子公司存货可变现净值，少计提存货跌价准备和资产减值损失4.05亿元，导致错报。
